In [6]:
import xgboost as xgb

import pandas as pd
import numpy as np
import math

from sklearn.preprocessing import normalize, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from hyperopt import fmin, tpe, hp, STATUS_OK,Trials

In [7]:
def go_back(look_back, df):
    #Create previous Day price column 
    
    for i in range(look_back):
        name = 'Previous_Day_Price' + str(i)
        df[name] = df['Weighted_Price'].shift(+1+i)
        df.dropna(how='any', inplace=True)
        
    return df

In [8]:
df = pd.read_csv('fulldata.csv')
#Drop np.nan
df.dropna(how='any', inplace=True)

df = df[['Weighted_Price', 'Polarity_Textblob','Trend_Values', 'Polarity_Vader']]

#Go back
look_back = 1
df = go_back(look_back, df)

df = df.values

# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
df = scaler.fit_transform(df)

dataset = np.delete(df,(0), axis=1)
labels = np.delete(df,(1,2,3,4,5,6,7,8,9), axis=1).ravel()

x_train, x_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2, shuffle=False)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, shuffle = False)
    
eval_set = [(x_val, y_val)]

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: in the future out of bounds indices will raise an error instead of being ignored by `numpy.delete`.


In [9]:
def hyperopt_train_test(params):

    clf = xgb.XGBRegressor(**params)
    clf.fit(x_train, y_train, early_stopping_rounds=3, eval_metric="rmse", eval_set=eval_set, verbose=True)
    
    prediction = clf.predict(x_train)
    print (math.sqrt(mean_squared_error(prediction,y_train)) )
   
    return math.sqrt(mean_squared_error(prediction,y_train))

best = 200
def f(params):
    global best
    acc = hyperopt_train_test(params)
    if acc < best:
        best = acc
        print ('new best:', best, params)
    return {'loss': acc, 'status': STATUS_OK}

GradientBoosting4space = {
    
    'booster' : 'gbtree',
    'eval_metric' : 'rmse',
    'tree_method' : 'gpu_hist',
    'early_stopping_round' : 10,
    'grow_policy' : 'lossguide',
    'random_state' : 1234,
    'n_jobs' : -1,
    
    
    'learning_rate' : hp.quniform('learning_rate',0.01,1,0.01),
    'min_child_weight' : hp.quniform('min_child_weight',1,6,1),
    'subsample': hp.quniform('subsample', 0.3, 0.9, 0.1),
    'gamma' : hp.quniform('gamma',0.01,15,0.01),
    'lambda': hp.quniform('lambda', 0.1, 1, 0.1),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.3, 0.99, 0.1),
    'max_depth': hp.choice('max_depth',[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14,15]), 

}

In [10]:
trials = Trials()
best = fmin(f, GradientBoosting4space , algo=tpe.suggest, max_evals=1000, trials=trials)
print ('best:')
print (best)

[0]	validation_0-rmse:0.069449
Will train until validation_0-rmse hasn't improved in 3 rounds.
[1]	validation_0-rmse:0.04266
[2]	validation_0-rmse:0.037012
[3]	validation_0-rmse:0.036769
[4]	validation_0-rmse:0.039432
[5]	validation_0-rmse:0.037865
[6]	validation_0-rmse:0.037839
Stopping. Best iteration:
[3]	validation_0-rmse:0.036769

0.037098451134885656
new best: 0.037098451134885656 {'booster': 'gbtree', 'colsample_bytree': 0.6000000000000001, 'early_stopping_round': 10, 'eval_metric': 'rmse', 'gamma': 8.47, 'grow_policy': 'lossguide', 'lambda': 0.7000000000000001, 'learning_rate': 0.58, 'max_depth': 13, 'min_child_weight': 3.0, 'n_jobs': -1, 'random_state': 1234, 'subsample': 0.8, 'tree_method': 'gpu_hist'}
[0]	validation_0-rmse:0.134885
Will train until validation_0-rmse hasn't improved in 3 rounds.
[1]	validation_0-rmse:0.130081
[2]	validation_0-rmse:0.127845
[3]	validation_0-rmse:0.125576
[4]	validation_0-rmse:0.120043
[5]	validation_0-rmse:0.117628
[6]	validation_0-rmse:0.1154

Will train until validation_0-rmse hasn't improved in 3 rounds.
[1]	validation_0-rmse:0.106837
[2]	validation_0-rmse:0.09599
[3]	validation_0-rmse:0.087249
[4]	validation_0-rmse:0.077705
[5]	validation_0-rmse:0.068821
[6]	validation_0-rmse:0.06186
[7]	validation_0-rmse:0.055937
[8]	validation_0-rmse:0.051456
[9]	validation_0-rmse:0.047071
[10]	validation_0-rmse:0.043874
[11]	validation_0-rmse:0.04098
[12]	validation_0-rmse:0.036648
[13]	validation_0-rmse:0.033914
[14]	validation_0-rmse:0.030857
[15]	validation_0-rmse:0.028844
[16]	validation_0-rmse:0.02628
[17]	validation_0-rmse:0.02408
[18]	validation_0-rmse:0.023336
[19]	validation_0-rmse:0.022165
[20]	validation_0-rmse:0.021612
[21]	validation_0-rmse:0.021245
[22]	validation_0-rmse:0.021149
[23]	validation_0-rmse:0.020501
[24]	validation_0-rmse:0.020413
[25]	validation_0-rmse:0.020328
[26]	validation_0-rmse:0.020148
[27]	validation_0-rmse:0.020332
[28]	validation_0-rmse:0.020537
[29]	validation_0-rmse:0.020472
Stopping. Best iterati

[2]	validation_0-rmse:0.027297
[3]	validation_0-rmse:0.030794
[4]	validation_0-rmse:0.029723
[5]	validation_0-rmse:0.028127
Stopping. Best iteration:
[2]	validation_0-rmse:0.027297

0.02069456211396636
[0]	validation_0-rmse:0.076907
Will train until validation_0-rmse hasn't improved in 3 rounds.
[1]	validation_0-rmse:0.04499
[2]	validation_0-rmse:0.033913
[3]	validation_0-rmse:0.030359
[4]	validation_0-rmse:0.025802
[5]	validation_0-rmse:0.02395
[6]	validation_0-rmse:0.023907
[7]	validation_0-rmse:0.024799
[8]	validation_0-rmse:0.025357
[9]	validation_0-rmse:0.026603
Stopping. Best iteration:
[6]	validation_0-rmse:0.023907

0.01906687268358553
[0]	validation_0-rmse:0.069445
Will train until validation_0-rmse hasn't improved in 3 rounds.
[1]	validation_0-rmse:0.034284
[2]	validation_0-rmse:0.035513
[3]	validation_0-rmse:0.035889
[4]	validation_0-rmse:0.035247
Stopping. Best iteration:
[1]	validation_0-rmse:0.034284

0.10915982802887644
[0]	validation_0-rmse:0.04987
Will train until vali

[16]	validation_0-rmse:0.02365
[17]	validation_0-rmse:0.023781
[18]	validation_0-rmse:0.023675
[19]	validation_0-rmse:0.023697
Stopping. Best iteration:
[16]	validation_0-rmse:0.02365

0.007172775896463913
[0]	validation_0-rmse:0.060792
Will train until validation_0-rmse hasn't improved in 3 rounds.
[1]	validation_0-rmse:0.042038
[2]	validation_0-rmse:0.038733
[3]	validation_0-rmse:0.036669
[4]	validation_0-rmse:0.038678
[5]	validation_0-rmse:0.038149
[6]	validation_0-rmse:0.037947
Stopping. Best iteration:
[3]	validation_0-rmse:0.036669

0.028822689135837295
[0]	validation_0-rmse:0.034228
Will train until validation_0-rmse hasn't improved in 3 rounds.
[1]	validation_0-rmse:0.037752
[2]	validation_0-rmse:0.037492
[3]	validation_0-rmse:0.04124
Stopping. Best iteration:
[0]	validation_0-rmse:0.034228

0.03731225214477639
[0]	validation_0-rmse:0.093288
Will train until validation_0-rmse hasn't improved in 3 rounds.
[1]	validation_0-rmse:0.041209
[2]	validation_0-rmse:0.039232
[3]	validati

[3]	validation_0-rmse:0.03845
[4]	validation_0-rmse:0.032172
[5]	validation_0-rmse:0.028887
[6]	validation_0-rmse:0.024577
[7]	validation_0-rmse:0.02402
[8]	validation_0-rmse:0.024469
[9]	validation_0-rmse:0.02424
[10]	validation_0-rmse:0.024214
Stopping. Best iteration:
[7]	validation_0-rmse:0.02402

0.009759065684810943
[0]	validation_0-rmse:0.066082
Will train until validation_0-rmse hasn't improved in 3 rounds.
[1]	validation_0-rmse:0.038342
[2]	validation_0-rmse:0.040796
[3]	validation_0-rmse:0.041002
[4]	validation_0-rmse:0.041781
Stopping. Best iteration:
[1]	validation_0-rmse:0.038342

0.10220063436600746
[0]	validation_0-rmse:0.127808
Will train until validation_0-rmse hasn't improved in 3 rounds.
[1]	validation_0-rmse:0.119071
[2]	validation_0-rmse:0.110862
[3]	validation_0-rmse:0.103251
[4]	validation_0-rmse:0.09564
[5]	validation_0-rmse:0.090056
[6]	validation_0-rmse:0.083982
[7]	validation_0-rmse:0.078267
[8]	validation_0-rmse:0.073564
[9]	validation_0-rmse:0.069178
[10]	v

...

[15]	validation_0-rmse:0.02323
Stopping. Best iteration:
[12]	validation_0-rmse:0.02296

0.004008520142326884
[0]	validation_0-rmse:0.085071
Will train until validation_0-rmse hasn't improved in 3 rounds.
[1]	validation_0-rmse:0.055421
[2]	validation_0-rmse:0.03965
[3]	validation_0-rmse:0.033146
[4]	validation_0-rmse:0.027085
[5]	validation_0-rmse:0.02585
[6]	validation_0-rmse:0.02509
[7]	validation_0-rmse:0.024209
[8]	validation_0-rmse:0.024482
[9]	validation_0-rmse:0.0243
[10]	validation_0-rmse:0.024437
Stopping. Best iteration:
[7]	validation_0-rmse:0.024209

0.008299808238716557
[0]	validation_0-rmse:0.107857
Will train until validation_0-rmse hasn't improved in 3 rounds.
[1]	validation_0-rmse:0.082149
[2]	validation_0-rmse:0.065749
[3]	validation_0-rmse:0.054549
[4]	validation_0-rmse:0.044119
[5]	validation_0-rmse:0.037155
[6]	validation_0-rmse:0.032252
[7]	validation_0-rmse:0.028685
[8]	validation_0-rmse:0.026502
[9]	validation_0-rmse:0.024706
[10]	validation_0-rmse:0.022526
[11]

0.07692791882471565
[0]	validation_0-rmse:0.09281
Will train until validation_0-rmse hasn't improved in 3 rounds.
[1]	validation_0-rmse:0.063818
[2]	validation_0-rmse:0.047455
[3]	validation_0-rmse:0.036289
[4]	validation_0-rmse:0.03028
[5]	validation_0-rmse:0.028111
[6]	validation_0-rmse:0.026659
[7]	validation_0-rmse:0.025216
[8]	validation_0-rmse:0.024302
[9]	validation_0-rmse:0.023779
[10]	validation_0-rmse:0.023177
[11]	validation_0-rmse:0.022827
[12]	validation_0-rmse:0.022986
[13]	validation_0-rmse:0.022847
[14]	validation_0-rmse:0.022884
Stopping. Best iteration:
[11]	validation_0-rmse:0.022827

0.01080391098787318
[0]	validation_0-rmse:0.08174
Will train until validation_0-rmse hasn't improved in 3 rounds.
[1]	validation_0-rmse:0.052027
[2]	validation_0-rmse:0.034573
[3]	validation_0-rmse:0.028402
[4]	validation_0-rmse:0.024747
[5]	validation_0-rmse:0.023911
[6]	validation_0-rmse:0.023174
[7]	validation_0-rmse:0.02247
[8]	validation_0-rmse:0.022643
[9]	validation_0-rmse:0.0226

0.008958694441985058
[0]	validation_0-rmse:0.078442
Will train until validation_0-rmse hasn't improved in 3 rounds.
[1]	validation_0-rmse:0.048697
[2]	validation_0-rmse:0.032551
[3]	validation_0-rmse:0.027341
[4]	validation_0-rmse:0.024412
[5]	validation_0-rmse:0.023937
[6]	validation_0-rmse:0.023777
[7]	validation_0-rmse:0.023508
[8]	validation_0-rmse:0.023442
[9]	validation_0-rmse:0.023118
[10]	validation_0-rmse:0.023396
[11]	validation_0-rmse:0.023225
[12]	validation_0-rmse:0.023401
Stopping. Best iteration:
[9]	validation_0-rmse:0.023118

0.003698952614593417
[0]	validation_0-rmse:0.085071
Will train until validation_0-rmse hasn't improved in 3 rounds.
[1]	validation_0-rmse:0.055421
[2]	validation_0-rmse:0.039601
[3]	validation_0-rmse:0.033008
[4]	validation_0-rmse:0.027145
[5]	validation_0-rmse:0.026316
[6]	validation_0-rmse:0.025498
[7]	validation_0-rmse:0.024703
[8]	validation_0-rmse:0.024692
[9]	validation_0-rmse:0.024294
[10]	validation_0-rmse:0.023881
[11]	validation_0-rmse:0

[13]	validation_0-rmse:0.022932
[14]	validation_0-rmse:0.023021
Stopping. Best iteration:
[11]	validation_0-rmse:0.022592

0.0060714824702043915
[0]	validation_0-rmse:0.113686
Will train until validation_0-rmse hasn't improved in 3 rounds.
[1]	validation_0-rmse:0.091833
[2]	validation_0-rmse:0.076401
[3]	validation_0-rmse:0.064202
[4]	validation_0-rmse:0.054951
[5]	validation_0-rmse:0.049207
[6]	validation_0-rmse:0.042745
[7]	validation_0-rmse:0.038026
[8]	validation_0-rmse:0.034861
[9]	validation_0-rmse:0.031898
[10]	validation_0-rmse:0.028944
[11]	validation_0-rmse:0.02678
[12]	validation_0-rmse:0.024779
[13]	validation_0-rmse:0.023508
[14]	validation_0-rmse:0.022973
[15]	validation_0-rmse:0.022684
[16]	validation_0-rmse:0.022614
[17]	validation_0-rmse:0.022545
[18]	validation_0-rmse:0.022449
[19]	validation_0-rmse:0.022438
[20]	validation_0-rmse:0.022553
[21]	validation_0-rmse:0.022488
[22]	validation_0-rmse:0.02259
Stopping. Best iteration:
[19]	validation_0-rmse:0.022438

0.009956

[9]	validation_0-rmse:0.03071
[10]	validation_0-rmse:0.030663
[11]	validation_0-rmse:0.030728
Stopping. Best iteration:
[8]	validation_0-rmse:0.028379

0.02187084497593152
[0]	validation_0-rmse:0.098654
Will train until validation_0-rmse hasn't improved in 3 rounds.
[1]	validation_0-rmse:0.071541
[2]	validation_0-rmse:0.054529
[3]	validation_0-rmse:0.043072
[4]	validation_0-rmse:0.036128
[5]	validation_0-rmse:0.031988
[6]	validation_0-rmse:0.027441
[7]	validation_0-rmse:0.025714
[8]	validation_0-rmse:0.02454
[9]	validation_0-rmse:0.024115
[10]	validation_0-rmse:0.023699
[11]	validation_0-rmse:0.022735
[12]	validation_0-rmse:0.022543
[13]	validation_0-rmse:0.022464
[14]	validation_0-rmse:0.022398
[15]	validation_0-rmse:0.022322
[16]	validation_0-rmse:0.0223
[17]	validation_0-rmse:0.022266
[18]	validation_0-rmse:0.02231
[19]	validation_0-rmse:0.022313
[20]	validation_0-rmse:0.022283
Stopping. Best iteration:
[17]	validation_0-rmse:0.022266

0.002191863392352587
[0]	validation_0-rmse:0.07